# Cut Out UTRs
Code to cut out the UTRs out of the sequences of the intergenic sequences

## Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

<a id="help"></a>
### General Helper Functions

In [2]:
def rev_comp(seq):
    '''
    Switches a seqence from top to bottom 
    (ie 'ATGC' becomes 'TACG')
    
    Args:
        seq: a string which represents the top strand of DNA.
        Should only have the characters ATGC
    Ret:
        string sequence that represents the bottom sequence
        (all Ts become As, As become Ts, Gs become Cs, Cs become Gs)
        and is revered
        Resulting string will be all uppercase
    '''
    res = seq.lower()
    res = res.replace('t', 'A')
    res = res.replace('a', 'T')
    res = res.replace('g', 'C')
    res = res.replace('c', 'G')
    res = res[::-1]
    return res

def comp(seq):
    '''
    Returns the compliment of this sequence
    (all Ts become As, As become Ts, Gs become Cs, Cs become Gs)
    Us all uppercase
    '''
    res = seq.lower()
    res = res.replace('t', 'A')
    res = res.replace('a', 'T')
    res = res.replace('g', 'C')
    res = res.replace('c', 'G')
    return res

def rev(seq):
    '''
    reverses a string
    '''
    res = seq[::-1]
    return res

<a id="load"></a>
### Loading in Data
We load in our two data sets.

- df_intergenic contains the data we cleaned regarding the intergenic sequences
- df_TSS contains TSS positions and UTR lengths

In [3]:
df_intergenic = pd.read_excel('./source/Upstream-Sequences_df.xls')
df_TSS = pd.read_excel('./source/journal.pgen.1002867.s007.XLSX')

In [4]:
#df_intergenic contains data that we cleaned regarding intergenic sequences
df_intergenic.head()

,Unnamed: 0,IS Number,Gene,Upstream Sequence,L_END,R_END,Length,Adjacent Gene,Orientation
0,0,3,yaaX,ATCTATTCATTATCTCAATCAGGCCGGGTTTGCTTTTATGCAGCCC...,5021,5233,213,thrC,Codirectional+
1,1,5,yaaA,ATCACAACTATCGATCAACTCATTCTCATTTTTTGCTAAAGTCGGC...,6460,6528,69,yaaJ,Codirectional-
2,2,6,yaaJ,TGATAGTATTTCTCTTTAAACAGCTTGTTAGGGGGATGTAACCGGT...,7960,8237,278,talB,Divergent
3,3,6,talB,CATATCCCTCTTATTGCCGGTCGCGATGACTTTCCTGTGTAAACGT...,7960,8237,278,yaaJ,Divergent
4,4,7,mog,CATTCTTAGCGTGACCGGGAAGTCGGTCACGCTACCTCTTCTGAAG...,9192,9305,114,talB,Codirectional+


In [5]:
#df_TSS contains data regarding TSS positions and UTR lengths
df_TSS.head()

,Chromosome,TSS position,TSS reads,Strand,Gene ID,Gene name,Gene product,COG function,Gene start,Gene end,5' UTR length,Sequence -50 nt upstream + TSS (51 nt)
0,NC_000913,148,1475,+,b0001,thrL,thr operon leader peptide,Not in COGs,190,255,42,AATTAAAATTTTATTGACTTAGGTCACTAAATACTTTAACCAATAT...
1,NC_000913,195,11,+,b0002,thrA,aspartate kinase / homoserine dehydrogenase,Amino acid transport and metabolism,337,2799,142,GGCATAGCGCACAGACAGATAAAAATTACAGAGTACACAACATCCA...
2,NC_000913,2391,3,+,b0003,thrB,homoserine kinase,Amino acid transport and metabolism,2801,3733,410,GACCCGCGAGATGATCTTTCTGGTATGGATGTGGCGCGTAAACTAT...
3,NC_000913,2693,6,+,b0003,thrB,homoserine kinase,Amino acid transport and metabolism,2801,3733,108,CAAAGTGAAAAATGGCGAAAACGCCCTGGCCTTCTATAGCCACTAT...
4,NC_000913,2742,4,+,b0003,thrB,homoserine kinase,Amino acid transport and metabolism,2801,3733,59,CAGCCGCTGCCGTTGGTACTGCGCGGATATGGTGCGGGCAATGACG...


---
<a id="clean"></a>
## Cleaning Data
<a id="merge"></a>
### Merging datasets

In [6]:
# merged dataframe, merging on the Gene names
df_merged = df_intergenic.merge(df_TSS, left_on='Gene', right_on='Gene name')
# df_merged_cut = df_merged_cut.drop("Gene name", axis=1)
df_merged.head()

,Unnamed: 0,IS Number,Gene,Upstream Sequence,L_END,R_END,Length,Adjacent Gene,Orientation,Chromosome,...,TSS reads,Strand,Gene ID,Gene name,Gene product,COG function,Gene start,Gene end,5' UTR length,Sequence -50 nt upstream + TSS (51 nt)
0,0,3,yaaX,ATCTATTCATTATCTCAATCAGGCCGGGTTTGCTTTTATGCAGCCC...,5021,5233,213,thrC,Codirectional+,NC_000913,...,2,+,b0005,yaaX,predicted protein,Not in COGs,5234,5530,448,ACCACGCAACAGACAATGCGTGAGTTAAAAGAACTGGGCTACACTT...
1,0,3,yaaX,ATCTATTCATTATCTCAATCAGGCCGGGTTTGCTTTTATGCAGCCC...,5021,5233,213,thrC,Codirectional+,NC_000913,...,2,+,b0005,yaaX,predicted protein,Not in COGs,5234,5530,354,ATCCAGGCGAATATGGCTTGTTCCTCGGCACCGCGCATCCGGCGAA...
2,0,3,yaaX,ATCTATTCATTATCTCAATCAGGCCGGGTTTGCTTTTATGCAGCCC...,5021,5233,213,thrC,Codirectional+,NC_000913,...,4,+,b0005,yaaX,predicted protein,Not in COGs,5234,5530,346,GAATATGGCTTGTTCCTCGGCACCGCGCATCCGGCGAAATTTAAAG...
3,0,3,yaaX,ATCTATTCATTATCTCAATCAGGCCGGGTTTGCTTTTATGCAGCCC...,5021,5233,213,thrC,Codirectional+,NC_000913,...,4,+,b0005,yaaX,predicted protein,Not in COGs,5234,5530,303,AAGAGAGCGTGGAAGCGATTCTCGGTGAAACGTTGGATCTGCCAAA...
4,2,6,yaaJ,TGATAGTATTTCTCTTTAAACAGCTTGTTAGGGGGATGTAACCGGT...,7960,8237,278,talB,Divergent,NC_000913,...,15,-,b0007,yaaJ,YaaJ alanine AGSS transporter,Amino acid transport and metabolism,6529,7959,209,CAGCTTGTTAGGGGGATGTAACCGGTCTGCCCTGATGATATCACGA...


In [7]:
len(df_merged), len(df_intergenic), len(df_TSS)

(2058, 2520, 3746)

<a id="cut"></a>
### Cutting UTRs
We cut the UTRs for sequence

In [8]:
def cut_utr(ups_seq, utr_len, strand):
    # if the utr_len is > ups_seq len, throw it out
    if utr_len > len(ups_seq):
        return ''
    
    # there are different off by one erros (presented by
    # the data) depending on which strand we use
    if strand == '+':
        res = ups_seq[0:len(ups_seq) - utr_len - 1]
    if strand == '-':
        res = ups_seq[0:len(ups_seq) - utr_len + 1]
    return res

df_merged_cut = df_merged.copy()
df_merged_cut['Sequence without UTR'] = df_merged.apply(
    lambda row: cut_utr(row['Upstream Sequence'], row['5\' UTR length'], row['Strand']), axis=1)

df_merged_cut.head()

,Unnamed: 0,IS Number,Gene,Upstream Sequence,L_END,R_END,Length,Adjacent Gene,Orientation,Chromosome,...,Strand,Gene ID,Gene name,Gene product,COG function,Gene start,Gene end,5' UTR length,Sequence -50 nt upstream + TSS (51 nt),Sequence without UTR
0,0,3,yaaX,ATCTATTCATTATCTCAATCAGGCCGGGTTTGCTTTTATGCAGCCC...,5021,5233,213,thrC,Codirectional+,NC_000913,...,+,b0005,yaaX,predicted protein,Not in COGs,5234,5530,448,ACCACGCAACAGACAATGCGTGAGTTAAAAGAACTGGGCTACACTT...,
1,0,3,yaaX,ATCTATTCATTATCTCAATCAGGCCGGGTTTGCTTTTATGCAGCCC...,5021,5233,213,thrC,Codirectional+,NC_000913,...,+,b0005,yaaX,predicted protein,Not in COGs,5234,5530,354,ATCCAGGCGAATATGGCTTGTTCCTCGGCACCGCGCATCCGGCGAA...,
2,0,3,yaaX,ATCTATTCATTATCTCAATCAGGCCGGGTTTGCTTTTATGCAGCCC...,5021,5233,213,thrC,Codirectional+,NC_000913,...,+,b0005,yaaX,predicted protein,Not in COGs,5234,5530,346,GAATATGGCTTGTTCCTCGGCACCGCGCATCCGGCGAAATTTAAAG...,
3,0,3,yaaX,ATCTATTCATTATCTCAATCAGGCCGGGTTTGCTTTTATGCAGCCC...,5021,5233,213,thrC,Codirectional+,NC_000913,...,+,b0005,yaaX,predicted protein,Not in COGs,5234,5530,303,AAGAGAGCGTGGAAGCGATTCTCGGTGAAACGTTGGATCTGCCAAA...,
4,2,6,yaaJ,TGATAGTATTTCTCTTTAAACAGCTTGTTAGGGGGATGTAACCGGT...,7960,8237,278,talB,Divergent,NC_000913,...,-,b0007,yaaJ,YaaJ alanine AGSS transporter,Amino acid transport and metabolism,6529,7959,209,CAGCTTGTTAGGGGGATGTAACCGGTCTGCCCTGATGATATCACGA...,TGATAGTATTTCTCTTTAAACAGCTTGTTAGGGGGATGTAACCGGT...


**Note: Cases where UTR len > upstream sequence len.** For some genes at UTRs, the 5' UTR length is longer than the actual upstream sequence. These therefore have no sequence after we cut out the UTR and so must be dropped from the data set.

In [9]:
df_merged_cut['Length of Sequence Without UTR'] = df_merged_cut['Sequence without UTR'].apply(len)
mask1 = df_merged_cut['Length of Sequence Without UTR'] > 0

df_merged_cut = df_merged_cut[mask1]
df_merged_cut = df_merged_cut.drop(columns="Length of Sequence Without UTR")


print(len(df_merged_cut))
df_merged_cut.head()

1394


,Unnamed: 0,IS Number,Gene,Upstream Sequence,L_END,R_END,Length,Adjacent Gene,Orientation,Chromosome,...,Strand,Gene ID,Gene name,Gene product,COG function,Gene start,Gene end,5' UTR length,Sequence -50 nt upstream + TSS (51 nt),Sequence without UTR
4,2,6,yaaJ,TGATAGTATTTCTCTTTAAACAGCTTGTTAGGGGGATGTAACCGGT...,7960,8237,278,talB,Divergent,NC_000913,...,-,b0007,yaaJ,YaaJ alanine AGSS transporter,Amino acid transport and metabolism,6529,7959,209,CAGCTTGTTAGGGGGATGTAACCGGTCTGCCCTGATGATATCACGA...,TGATAGTATTTCTCTTTAAACAGCTTGTTAGGGGGATGTAACCGGT...
6,3,6,talB,CATATCCCTCTTATTGCCGGTCGCGATGACTTTCCTGTGTAAACGT...,7960,8237,278,yaaJ,Divergent,NC_000913,...,+,b0008,talB,transaldolase B,Carbohydrate transport and metabolism,8238,9191,47,ACCGTCTTGTCGGCGGTTGCGCTGACGTTGCGTCGTGATATCATCA...,CATATCCCTCTTATTGCCGGTCGCGATGACTTTCCTGTGTAAACGT...
8,6,11,dnaK,AGGATTCTCTTAGTGGGAAGAGGTAGGGGGATGAATACCCACTAGT...,11787,12162,376,yaaI,Divergent,NC_000913,...,+,b0014,dnaK,"chaperone Hsp70, DNA biosynthesis, autoregulat...","Posttranslational modification, protein turnov...",12163,14079,6,CCCCTATTACAGACTCACAACCACATGATGACCGAATATATAGTGG...,AGGATTCTCTTAGTGGGAAGAGGTAGGGGGATGAATACCCACTAGT...
11,10,17,nhaA,TATCTGCCGTTCAGCTAATGCCTGAGACAGACAGCCTCAAGCACCC...,17007,17488,482,sokC,Codirectional+,NC_000913,...,+,b0019,nhaA,NhaA sodium/proton transporter,Inorganic ion transport and metabolism,17489,18655,265,TCATCGCAATTATTGACGACAAGCTGGATTATTTTTGAAATATTGG...,TATCTGCCGTTCAGCTAATGCCTGAGACAGACAGCCTCAAGCACCC...
12,12,22,rpsT,TTAACGGCGCTTATTTGCACAAATCCATTGACAAAAGAAGGCTAAA...,21079,21180,102,yaaY,Divergent,NC_000913,...,-,b0023,rpsT,30S ribosomal subunit protein S20,Translation,20815,21078,40,TTTGCACAAATCCATTGACAAAAGAAGGCTAAAAGGGCATATTCCT...,TTAACGGCGCTTATTTGCACAAATCCATTGACAAAAGAAGGCTAAA...


<a id="div"></a>
### Divergent Case

The divergent case is a lot trickier. We need to cut off the UTRs from both sides (\*\* NOTE: do not forget about the off by one error) for each TSS. This will create a lot more entries in our table. We also need to add another column which denotes the side of the TSS site of the other side that got cut.

First we just grab the entries with divergent orientation

In [10]:
df_div = df_merged_cut[df_merged_cut['Orientation'] == 'Divergent']
# df_div = df_div[(df_div['Gene'] == 'yaaJ') | (df_div['Gene'] == 'talB')] # uncomment to see simple case
df_div.head()

,Unnamed: 0,IS Number,Gene,Upstream Sequence,L_END,R_END,Length,Adjacent Gene,Orientation,Chromosome,...,Strand,Gene ID,Gene name,Gene product,COG function,Gene start,Gene end,5' UTR length,Sequence -50 nt upstream + TSS (51 nt),Sequence without UTR
4,2,6,yaaJ,TGATAGTATTTCTCTTTAAACAGCTTGTTAGGGGGATGTAACCGGT...,7960,8237,278,talB,Divergent,NC_000913,...,-,b0007,yaaJ,YaaJ alanine AGSS transporter,Amino acid transport and metabolism,6529,7959,209,CAGCTTGTTAGGGGGATGTAACCGGTCTGCCCTGATGATATCACGA...,TGATAGTATTTCTCTTTAAACAGCTTGTTAGGGGGATGTAACCGGT...
6,3,6,talB,CATATCCCTCTTATTGCCGGTCGCGATGACTTTCCTGTGTAAACGT...,7960,8237,278,yaaJ,Divergent,NC_000913,...,+,b0008,talB,transaldolase B,Carbohydrate transport and metabolism,8238,9191,47,ACCGTCTTGTCGGCGGTTGCGCTGACGTTGCGTCGTGATATCATCA...,CATATCCCTCTTATTGCCGGTCGCGATGACTTTCCTGTGTAAACGT...
8,6,11,dnaK,AGGATTCTCTTAGTGGGAAGAGGTAGGGGGATGAATACCCACTAGT...,11787,12162,376,yaaI,Divergent,NC_000913,...,+,b0014,dnaK,"chaperone Hsp70, DNA biosynthesis, autoregulat...","Posttranslational modification, protein turnov...",12163,14079,6,CCCCTATTACAGACTCACAACCACATGATGACCGAATATATAGTGG...,AGGATTCTCTTAGTGGGAAGAGGTAGGGGGATGAATACCCACTAGT...
12,12,22,rpsT,TTAACGGCGCTTATTTGCACAAATCCATTGACAAAAGAAGGCTAAA...,21079,21180,102,yaaY,Divergent,NC_000913,...,-,b0023,rpsT,30S ribosomal subunit protein S20,Translation,20815,21078,40,TTTGCACAAATCCATTGACAAAAGAAGGCTAAAAGGGCATATTCCT...,TTAACGGCGCTTATTTGCACAAATCCATTGACAAAAGAAGGCTAAA...
20,24,53,lptD,TATATTCCCCAAATCGACACACGGATATCAGGGCTATCTCCCACAA...,57110,57363,254,djlA,Divergent,NC_000913,...,-,b0054,lptD,outer membrane LPS assembly complex,Cell wall/membrane biogenesis,54755,57109,47,GGACAGGATTAACACTAGCGTAGAGATGACGAGTACGTTAGTCTCT...,TATATTCCCCAAATCGACACACGGATATCAGGGCTATCTCCCACAA...


In [11]:
df_div[(df_div['Gene'] == 'talB') | (df_div['Gene'] == 'yaaJ')]

,Unnamed: 0,IS Number,Gene,Upstream Sequence,L_END,R_END,Length,Adjacent Gene,Orientation,Chromosome,...,Strand,Gene ID,Gene name,Gene product,COG function,Gene start,Gene end,5' UTR length,Sequence -50 nt upstream + TSS (51 nt),Sequence without UTR
4,2,6,yaaJ,TGATAGTATTTCTCTTTAAACAGCTTGTTAGGGGGATGTAACCGGT...,7960,8237,278,talB,Divergent,NC_000913,...,-,b0007,yaaJ,YaaJ alanine AGSS transporter,Amino acid transport and metabolism,6529,7959,209,CAGCTTGTTAGGGGGATGTAACCGGTCTGCCCTGATGATATCACGA...,TGATAGTATTTCTCTTTAAACAGCTTGTTAGGGGGATGTAACCGGT...
6,3,6,talB,CATATCCCTCTTATTGCCGGTCGCGATGACTTTCCTGTGTAAACGT...,7960,8237,278,yaaJ,Divergent,NC_000913,...,+,b0008,talB,transaldolase B,Carbohydrate transport and metabolism,8238,9191,47,ACCGTCTTGTCGGCGGTTGCGCTGACGTTGCGTCGTGATATCATCA...,CATATCCCTCTTATTGCCGGTCGCGATGACTTTCCTGTGTAAACGT...


Next we cut the front ends off of the sequences, and store the adjacent genes TSS that was used to cut off the front of the sequence

In [12]:
# function for cutting the front of the utrs for the divergent case
def cut_utr_front(ups_seq, utr_len, strand):
    # throw it out if it would just cut the whole thing up
    if utr_len > len(ups_seq):
        return ''
    
    # off by one error, change the end based on
    # strand direction of the other strand
    if strand == "+":
        res = ups_seq[utr_len+1:]
    else:
        res = ups_seq[utr_len-1:]
    return res

This part takes a while, hence the Run variable. Set it to True if you need to run the code, otherwise it will load in already existing data.

In [ ]:
Run = True
pd.options.mode.chained_assignment = None  # default='warn'

num_missing_adj_entries = 0

if Run:
    div_lst = df_div.to_dict('records')  # a list of dictionaries of the divergent entries, easier to iterate

    div_exp = pd.DataFrame()  # data structure to store expanded entries

    for r in div_lst:
        # get neccesary data
        gene = r['Gene']
        utr_len = r['5\' UTR length']
        adj_gene = r['Adjacent Gene']
        TSS = r['TSS position']  
        strand = r['Strand']


        # get the rows that have the adjacent gene
        adj_genes = df_div[df_div['Gene'] == adj_gene]
        if len(adj_genes) == 0:
            num_missing_adj_entries += 1
        adj_genes['Sequence without UTR'] = df_merged_cut.apply(
            lambda row: cut_utr_front(row['Sequence without UTR'], utr_len, strand), axis=1)                                                    
        adj_genes['adj_TSS'] = TSS
        div_exp = pd.concat([div_exp, adj_genes])       
    div_exp = div_exp.dropna()
    div_exp.to_excel("./export/div_exp_int.xlsx")
else:
    div_exp =  pd.read_excel("./export/div_exp_int.xlsx")
div_exp.head()    

In [ ]:
# 1 - num_missing_adj_entries/len(div_lst), 57, "percent of entries lost because adjacent gene was missing"

Drop all entries where there no longer is a sequence

In [ ]:
def mod_len(string):
    if type(string) == float:
        return 0
    else:
        return len(string)

In [ ]:
# remove entries without the length
print(len(div_exp))
div_exp['Length of Sequence Without UTR'] = div_exp['Sequence without UTR'].apply(mod_len)
mask1 = div_exp['Length of Sequence Without UTR'] > 0

div_exp = div_exp[mask1]
div_exp = div_exp.drop(columns="Length of Sequence Without UTR")
print(len(div_exp))
# div_exp

In [ ]:
1 - 457/525, "divergent genes lost because both ends were cut off"

Concat this dataframe with all of our codirectional data entries, and sort by IS Number so they're in the same order as before. (I don't know if this is the final ordering we want everything in, but should be easy to change)

In [ ]:
df_co = df_merged_cut[df_merged_cut['Orientation'] != 'Divergent']
df_co["adj_TSS"] = np.nan
df_merged_cut_div = pd.concat([df_co, div_exp])

In [ ]:
df_merged_cut_div = df_merged_cut_div.sort_values(by=['IS Number'])
df_merged_cut_div.head()

In [ ]:
len(df_merged_cut_div), len(df_co), len(div_exp)

**Note: Data lost.** It seems likely that some sequences got cut off on both ends so that there was overlap and so no sequence covered by both genes. In addition, the adjacent gene for some genes was missing from the dataset (likely lost in a previous step or merge). An example of this is seen with the gene dnaK. It's adjacent gene is yaaI, which is not present in the data set.

<a id="reorder"></a>
### Reordering

Start by grabbing the columns we want

In [ ]:
df_reordered = df_merged_cut_div[['IS Number', 
             'Gene', 
             'Gene ID', 
             'Gene product', 
             'COG function', 
             'L_END', 
             'R_END', 
             'Length', 
             'Upstream Sequence', 
             'Orientation', 
             'Strand', 
             'TSS position', 
             'TSS reads', 
             'Sequence without UTR', 
             'Sequence -50 nt upstream + TSS (51 nt)', 
             '5\' UTR length',
             'Gene start',
             'Gene end',
             'Adjacent Gene',
             'adj_TSS'
            ]]
df_reordered.iloc[0:2]

Renaming the columns

In [ ]:
df_reordered = df_reordered.rename(index=str, 
                   columns={
                       "L_END": "IS Left End",
                       "R_END": "IS Right End",
                       "Length": "IS Length",
                       "Upstream Sequence": "Intergenic Sequence",
                       "Sequence without UTR": "Intergenic Sequence no UTR",
                       "adj_TSS": "Ajacent Gene TSS position"                      
                   })

df_reordered.iloc[0:2]

<a id="export"></a>
### Exporting

In [ ]:
df_reordered.to_excel("./20190717-IS-noUTRs-df1.xlsx")

---
<a id="test"></a>
## Tests
<a id="co_test"></a>
### Codirectional Case
For this case, we are just checking if the UTR got cut out correctly, checking it in benchling with ecocyc.

Things to test:
* Genes with a utr longer than the intergenic sequence were removed
* Actually cut correctly

In [ ]:
df_reordered.head()

In [ ]:
# all instances of yaaX have a longer 5' UTR than the intergenic sequence and so should have been removed
assert(len(df_reordered[df_reordered['Gene'] == 'yaaX']) == 0)

<a id="plus"></a>
#### Codirectional +: nhaA

Checking for a gene that did actually work (nhaA). Note that it should since the length of the intergenic sequence is 482 and the 5' UTR length is 265.0

In [ ]:
co = df_reordered[df_reordered['Orientation'] != 'Divergent']
co.head()
nhaA = df_reordered[df_reordered['Gene'] == 'nhaA']
nhaA

In [ ]:
ups_seq = nhaA['Intergenic Sequence'][0]
utr_len = nhaA['5\' UTR length'][0]
no_utr = nhaA['Intergenic Sequence no UTR'][0]
ups_seq, utr_len, no_utr

I then checked this in benchling.
* First I went to [ecocyc](https://ecocyc.org/gene?orgid=ECOLI&id=EG10652) to get the information for nhaA and pasted it into benchling
* Assuming that the 5' UTR length was correct, I annotated it and found where the Intergenic Sequence no UTR should end
* I then checked that the Intergenic Sequence no UTR generated by code matched

It did! Success for + codirectional case

Here is the benchling file I used for this test:

https://benchling.com/s/seq-3Cy6xOezeaXP4FwYMMGQ

Although this appears to be correct according to our information, the benchling data implies that this is the incorrect UTR length. On benchling it says the TSS should be at position 17317 or 17458 instead of 17224

In [ ]:
17224 - 17317 

<a id="minus"></a>
#### Codirectional -: polB
After starting to test the divergent case, I realized there are different problems that arise from the codirectional - case. These come from different off by 1 errros

In [ ]:
polB = co[co['Gene'] == 'polB']
polB

In [ ]:
ups_seq = polB['Intergenic Sequence'][0]
utr_len = polB['5\' UTR length'][0]
no_utr = polB['Intergenic Sequence no UTR'][0]
ups_seq, utr_len, no_utr

I then checked this in benchling.
* I got the information for [polB](https://ecocyc.org/gene?orgid=ECOLI&id=EG10652) and put it into benchling
* Assuming that the 5' UTR length was correct, I annotated it and found where the Intergenic Sequence no UTR should end
* I then checked that the Intergenic Sequence no UTR generated by code matched

Success for Codirectional -


Here is the benchling file I used for this test:

https://benchling.com/s/seq-IROLxENViRMKzNXJiLCI

<a id="div_test"></a>
### Divergent Case: talB and yaaJ

We'll look at talB and yaaJ (seeing if they were both done correctly). Since they each only have one valid TSS each, we will examine this case since it is simple. I have checked earlier to make sure the combinatorics were working (ie the case with multiple TSSs), hence why I will check this simple case

In [ ]:
div = df_reordered[df_reordered['Orientation'] == 'Divergent']
div.head()
talB = df_reordered[df_reordered['Gene'] == 'talB']
yaaJ = df_reordered[df_reordered['Gene'] == 'yaaJ']

In [ ]:
talB_ups_seq = talB['Intergenic Sequence'][0]
talB_utr_len = talB['5\' UTR length'][0]
talB_no_utr = talB['Intergenic Sequence no UTR'][0]

yaaJ_ups_seq = yaaJ['Intergenic Sequence'][0]
yaaJ_utr_len = yaaJ['5\' UTR length'][0]
yaaJ_no_utr = yaaJ['Intergenic Sequence no UTR'][0]

the Intergenic Sequence and Intergenic Sequence no UTR of both genes should have the same length and be reverse compliments of eachother. This should catch errors in the data and off by one errors

In [ ]:
assert(len(talB_ups_seq) == len(yaaJ_ups_seq)), "talB and yaaJ should have the same length of intergenic sequence"
assert(talB_ups_seq == rev_comp(yaaJ_ups_seq)), "talB's intergenic sequence should be the reverse compliment of yaaJ's intergenic sequence"
assert(len(yaaJ_no_utr) == len(talB_no_utr)), "talB and yaaJ should have the same length of Intergenic Sequence no UTR"
assert(yaaJ_no_utr == rev_comp(talB_no_utr)), "talB's intergenic sequence no UTR should be the reverse compliment of yaaJ's intergenic sequence no UTR"

In [ ]:
talB_no_utr, talB_utr_len, yaaJ_no_utr, yaaJ_utr_len, 

Finally we'll do a benchling check
* Got the ecocyc info for [talB](https://ecocyc.org/gene?orgid=ECOLI&id=EG11556) and [yaaJ](https://ecocyc.org/gene?orgid=ECOLI&id=EG11555)
* Assuming that the 5' UTR length was correct, I annotated them both.
* I then checked that the Intergenic Sequence no UTR generated by code matched

Here are the benchling files I used for this test:

talB: https://benchling.com/s/seq-84PLtGcoFBE43orWdOIf

yaaJ: https://benchling.com/s/seq-SdK8PctSCKrI96bWQMBo